### Hyperparams

In [5]:
import torch

batch_size = 32
block_size = 8
max_iters = 3_000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
train_split = 0.9
device = 'cuda'

n_embd = 32

### Data

In [2]:
with open ('/data/tinyshakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s] # encoder: map string to char ids
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: map integer ids to a string

data = torch.tensor(encode(text), dtype=torch.long)

# Splitting the dataset into training and validation sets
n = int(train_split * len(data)) # 90% training, 10% validation
train_data, validation_data = data[:n], data[n:]

def get_batch(split):
  # batch of data of inputs x and targets y
  data = train_data if split == 'train' else validation_data
  # array(batch_size) of random offsets within data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  # Context and target sequences
  # ? Why a single sequence is not used?
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  x, y = x.to(device), y.to(device)

  return x, y

### Loss

In [3]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()

  for split in ['train', 'validation']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()

    out[split] = losses.mean()

  model.train()
  return out

### Model

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()

    # Video says: each token directly reads off the logits for the next token
    # from the lookup table.
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    # idx and targets are (B, T) tensors of integers, Batch, Time
    # Token embedding table will return shape of (Batch, Time, Channel)
    # Batch - number of sequences
    # Time - number of tokens in each sequence
    # Channel - Embeddings vector for a token?
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      return logits, None

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)

    # Cross entropy expects input to be in a 2D tensor of shape (Batch*Time, Channel)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is a (Batch, Time) tensor of integers, representing current context
    for _ in range(max_new_tokens):
      # compute the predictions
      logits, loss = self(idx) # (B, T, C)

      # -1 makes very little sense for a bigram model,
      # as we're essentially throwing away everything except the very last token in a batch
      # to make our prediction.
      # This is done in such a way only to allow for easier transition to an N-gram model later.
      logits = logits[:, -1, :] # (B, C)

      # Probabilities from logits
      # Softmax ~ [1, 2, 3, 4]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

      # append to the currently running context
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
    return idx

model = BigramLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train

In [7]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f'Iter {iter}, Train loss: {losses["train"]}, Validation loss: {losses["validation"]}')

  # Sample batch of data
  xb, yb = get_batch('train')

  # Eval
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

Iter 0, Train loss: 4.362564563751221, Validation loss: 4.355724811553955
Iter 300, Train loss: 2.531529188156128, Validation loss: 2.5499892234802246
Iter 600, Train loss: 2.511695146560669, Validation loss: 2.54876446723938
Iter 900, Train loss: 2.4908628463745117, Validation loss: 2.522688388824463
Iter 1200, Train loss: 2.497490882873535, Validation loss: 2.510655164718628
Iter 1500, Train loss: 2.4841389656066895, Validation loss: 2.51910138130188
Iter 1800, Train loss: 2.473388671875, Validation loss: 2.5083813667297363
Iter 2100, Train loss: 2.498110294342041, Validation loss: 2.514561414718628
Iter 2400, Train loss: 2.486794948577881, Validation loss: 2.514545440673828
Iter 2700, Train loss: 2.486311197280884, Validation loss: 2.513540029525757


### Generate

In [8]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
batch = model.generate(ctx, max_new_tokens=500)
logits = batch[0].tolist()

print(decode(logits))


I
O: asafous ty t te othutomod tarnd he, preckn,
OHenthif o--wishelapiwistrs we s, orean,
TANGluprthand n,
KI Miced tlat mango win thy
bellaramean t.


DUKithapr me machan fl haisthe iere--sthurore cense wn ck me, woass t avest t, metha'elm; hy?



I he wlarBRGLONGofound s heve she, fthiran t anersamono be'sstore wend d ph, Rpefoue t he ososour talun I wibe g y stharinthed;
Whee s mir bakns t
Tou'esa amnfod bs as Y:
K:
BOf manase VO on my foristhatessotritorulerard haikn 'sealanks, g duthintide.
